# Matching Algorithm  Prototype

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from numpy.random import randn
from numpy.matlib import repmat

from scipy.stats import norm
from scipy.optimize import fmin
from scipy.special import erf

from patsy import dmatrices

from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LassoCV, RidgeClassifierCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
def try_float(v):
    try:
        return float(v)
    except Exception as e:
        return np.nan


In [3]:
def plot_roc(X_test, y_test, model):
    print(y_test.shape)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(y_test.shape[1] + 1):
        fpr[i], tpr[i], thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    plt.figure()
    lw = 2
    plt.plot(fpr[1], tpr[1], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [4]:
def array_vector(col):
    return np.array(str(col).split(' '), dtype=object)[0]

arrayerize = np.vectorize(array_vector)

def one_hot_encode(df, column, labels_column=None, whitelist=[]):
    # This is gross but since strings are iterable, we have to wrap them in a list
    # in order for the binarizer to parse the labels as strings and not chars
    labels = arrayerize(pd.DataFrame(df[column]))
    terms = arrayerize(pd.DataFrame(list(set(df[column]))))

    mlb = MultiLabelBinarizer()
    mlb.fit(terms)
    mlb.transform(labels)
    columns = [ f'{column}-{classname}' for classname in  mlb.classes_]

    encoded = pd.DataFrame(mlb.transform(labels), columns=columns, index=df[column].index)

    df.drop(column, axis=1, inplace=True)
    return df.join(encoded)

In [5]:
firms = pd.read_csv('../../match-data/match-v1/firms.csv')
jobs = pd.read_csv('../../match-data/match-v1/job-openings.csv')
jobs_parent = pd.read_csv('../../match-data/match-v1/job-openings-parent.csv')
job_seekers = pd.read_csv('../../match-data/match-v1/job-seekers.csv')
job_seekers = job_seekers[job_seekers['closed'] == False]
matches = pd.read_csv('../../match-data/match-v1/matches.csv')
matches_parent = pd.read_csv('../../match-data/match-v1/matches-parent.csv')

ignore = ['number', 'caseid', 'parent_case_id', 'job_id', 'employed', 'hired_yes_no', 'quit', 'fired']

job_seekers.columns = ['JS-' + c if c not in ignore else c for c in job_seekers.columns]
firms.columns = ['JOB-' + c if c not in ignore else c for c in firms.columns]
jobs.columns = ['JOB-' + c if c not in ignore else c for c in jobs.columns]
matches.columns = ['MATCH-' + c if c not in ignore else c for c in matches.columns]

matches = pd.merge(matches, matches_parent, on='number')
jobs = pd.merge(jobs, jobs_parent, on='number')
job_seekers['JS-caseid'] = job_seekers['caseid']
firms['FIRM-caseid'] = firms['caseid']
jobs['JOB-caseid'] = jobs['caseid']

In [6]:
def cartesian_product_simplified(left, right):
    la, lb = len(left), len(right)
    ia2, ib2 = np.broadcast_arrays(*np.ogrid[:la,:lb])

    return pd.DataFrame(
        np.column_stack([left.values[ia2.ravel()], right.values[ib2.ravel()]]))

In [7]:
categorical_columns = ["JS-gender", "JS-highest_edu_level", "JS-nationality", "JS-gendermix_not_allowed", "JS-benefit1", "JS-benefit2", "JS-english_proficiency", "JS-impairments", "JS-major", "JS-opposite_gender_coworkers", "JS-opposite_gender_manager", "JS-first_job_field_preference", "JS-second_job_field_preference", "JOB-bus_covered", "JOB-childcare_subsidy_offered", "JOB-dorm_covered", "JOB-driving_ability_required", "JOB-education_required", "JOB-english_proficiency_required", "JOB-female_requied", "JOB-free_meals_at_wok", "JOB-health_insurance_offered", "JOB-hearing_disability_accepted", "JOB-housing_subsidy_offered", "JOB-it_proficiency_required", "JOB-job_category", "JOB-job_description", "JOB-job_production", "JOB-jordanian_experience_required", "JOB-literacy_required", "JOB-male_required", "JOB-meal_subsidy_offered", "JOB-night_shifts_required", "JOB-noncognitive_skill_preference1", "JOB-noncognitive_skill_preference2", "JOB-numeracy_requied", "JOB-physical_disability_accepted", "JOB-physical_work_abilities_required", "JOB-problem_solving_required", "JOB-school_subsidy_offered", "JOB-specialization_required", "JOB-speech_disability_accepted", "JOB-syrian_considered", "JOB-transport_subsidy_offered", "JOB-visual_disability_accepted", "JOB-work_permit_offered",]
all_columns = ["JS-age", "JS-gender", "JS-highest_edu_level", "JS-will_work_night_shift", "JS-nationality", "JS-gendermix_not_allowed", "JS-will_work_qiz",  "JS-benefit1", "JS-benefit2", "JS-daily_hours_willing_to_work", "JS-days_willing_train_unpaid", "JS-distance_willing_to_travel", "JS-english_proficiency", "JS-experience_clerical_work", "JS-experience_factory", "JS-experience_management_work", "JS-experience_manual_labor", "JS-experience_professional_work", "JS-follow_up_agreement", "JS-impairments", "JS-major", "JS-nonarab_coworkers", "JS-opposite_gender_coworkers", "JS-opposite_gender_manager", "JS-weekly_days_willing_to_work", "JS-will_live_in_dorm", "JS-will_train_unpaid", "JS-years_education", "JS-years_exp", "JS-first_job_field_preference", "JS-rwage1", "JS-second_job_field_preference", "JOB-bus_covered", "JOB-childcare_subsidy_offered", "JOB-dorm_covered", "JOB-driving_ability_required", "JOB-education_required", "JOB-english_proficiency_required", "JOB-female_requied", "JOB-free_meals_at_wok", "JOB-health_insurance_offered", "JOB-hearing_disability_accepted", "JOB-housing_subsidy_offered", "JOB-it_proficiency_required", "JOB-job_category", "JOB-job_description", "JOB-job_production", "JOB-jordanian_experience_required", "JOB-literacy_required", "JOB-male_required", "JOB-meal_subsidy_offered", "JOB-night_shifts_required", "JOB-noncognitive_skill_preference1", "JOB-noncognitive_skill_preference2", "JOB-numeracy_requied", "JOB-physical_disability_accepted", "JOB-physical_work_abilities_required", "JOB-problem_solving_required", "JOB-school_subsidy_offered", "JOB-specialization_required", "JOB-speech_disability_accepted", "JOB-syrian_considered", "JOB-transport_subsidy_offered", "JOB-visual_disability_accepted", "JOB-wage_offered", "JOB-work_permit_offered", "JOB-years_experience_required", 'hired_yes_no', 'quit', 'fired']
scalar_columns = ["JS-age", "JS-daily_hours_willing_to_work", "JS-days_willing_train_unpaid", "JS-distance_willing_to_travel", "JS-years_education", "JS-years_exp", "JS-rwage1", "JS-num_children", "JS-personal_income", "JOB-wage_offered"]

## Filter out relevant columns


In [8]:
job_seekers.drop(['quit', 'employed', 'fired'], axis=1, inplace=True)
matches['parent_case_id'] = matches['parent_case_id']
job_seekers['parent_case_id'] = job_seekers['caseid']
merged = pd.merge(job_seekers, matches, on='parent_case_id')
merged = pd.merge(merged, jobs, on='job_id')

# Drop irrelevant columns that will throw off our predictions
merged = merged.drop(
    ["MATCH-interest_applying", "JS-city", "JOB-num_vacancies"],
    axis=1
)

merged['employed'] = merged['employed'].fillna(0)
merged['employed'] = merged['employed'].replace(['---'], 0)

merged['quit'] = merged['quit'].fillna(0)
merged['quit'] = merged['quit'].replace(['---'], 0)
merged['quit'] = merged['quit'].replace(['no'], 0)
merged['quit'] = merged['quit'].replace(['yes'], 1)

merged['fired'] = merged['fired'].fillna(0)
merged['fired'] = merged['fired'].replace(['---'], 0)
merged['fired'] = merged['fired'].replace(['no'], 0)
merged['fired'] = merged['fired'].replace(['yes'], 1)

merged['employed'] = merged['employed'].astype(bool)
merged['quit'] = merged['quit'].astype(bool)
merged['fired'] = merged['fired'].astype(bool)


In [9]:
merged = pd.merge(job_seekers, matches, on='parent_case_id')
merged = pd.merge(merged, jobs, on='job_id')

merged.to_csv('../../match-data/match-v1/merged.csv')

In [10]:
def cartesian_product_basic(left, right):
    return (
       left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1))

In [11]:
formatted = pd.DataFrame()

In [12]:
merged['employed'] = merged['employed'].fillna(0)
merged['employed'] = merged['employed'].replace(['---'], 0)

merged['quit'] = merged['quit'].fillna(0)
merged['quit'] = merged['quit'].replace(['---'], 0)
merged['quit'] = merged['quit'].replace(['no'], 0)
merged['quit'] = merged['quit'].replace(['yes'], 1)

merged['fired'] = merged['fired'].fillna(0)
merged['fired'] = merged['fired'].replace(['---'], 0)
merged['fired'] = merged['fired'].replace(['no'], 0)
merged['fired'] = merged['fired'].replace(['yes'], 1)

merged['employed'] = merged['employed'].astype(bool)
merged['quit'] = merged['quit'].astype(bool)
merged['fired'] = merged['fired'].astype(bool)

In [13]:
firms['parent_case_id'] = firms['caseid']
job_firm = pd.merge(jobs, firms, on="parent_case_id")
job_firm.info(memory_usage='deep')
job_seekers[job_seekers['JS-closed'] == False].info(memory_usage='deep')

jobs = jobs.filter(['JOB-job_id'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255 entries, 0 to 254
Columns: 530 entries, number_x to FIRM-caseid
dtypes: bool(2), float64(6), int64(4), object(518)
memory usage: 7.8 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1963 entries, 5 to 2879
Columns: 424 entries, JS-age to parent_case_id
dtypes: bool(1), int64(1), object(422)
memory usage: 48.6 MB


In [39]:
job_firm_js = cartesian_product_simplified(job_firm, job_seekers.sample(n=500))
job_firm_js.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127500 entries, 0 to 127499
Columns: 954 entries, 0 to 953
dtypes: object(954)
memory usage: 6.9 GB


In [42]:
job_firm_js.columns = list(job_firm.columns) + list(job_seekers.columns)

Index(['number_x', 'caseid_x', 'JOB-name_x', 'JOB-bus_covered',
       'JOB-childcare_subsidy_offered', 'JOB-dorm_covered',
       'JOB-driving_ability_required', 'JOB-education_required',
       'JOB-english_proficiency_required', 'JOB-female_requied',
       ...
       'JS-cjsect', 'JS-main_expense_other', 'JS-mspend13', 'JS-mspend3',
       'JS-mspend5', 'JS-method_id', 'JS-amount_withdrawn', 'JS-num_followups',
       'JS-caseid', 'parent_case_id'],
      dtype='object', length=954)

In [43]:
to_drop = job_firm_js.columns.difference(merged.columns)
display(to_drop)
job_firm_js.drop(to_drop, axis=1)
display(job_firm_js)

Index(['FIRM-caseid', 'JOB-adverstise_through_other',
       'JOB-advertise_through_agency', 'JOB-advertise_through_building_site',
       'JOB-advertise_through_epu', 'JOB-advertise_through_newspapers',
       'JOB-advertise_through_ngo', 'JOB-advertise_through_online',
       'JOB-advertise_through_social_media', 'JOB-age_client_services',
       ...
       'JOB-temporary_contract_professionals',
       'JOB-temporary_production_workers', 'JOB-temporary_support_services',
       'JOB-total_employees', 'JOB-training_not_required',
       'JOB-training_offered', 'JOB-training_type_offered', 'number__0',
       'number__1', 'parent_case_id'],
      dtype='object', length=464)

,number_x,caseid_x,JOB-name_x,JOB-bus_covered,JOB-childcare_subsidy_offered,JOB-dorm_covered,JOB-driving_ability_required,JOB-education_required,JOB-english_proficiency_required,JOB-female_requied,...,JS-cjsect,JS-main_expense_other,JS-mspend13,JS-mspend3,JS-mspend5,JS-method_id,JS-amount_withdrawn,JS-num_followups,JS-caseid,parent_case_id
0,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,NaN,---,---,---,---,---,NaN,0,3cad7ddf-2726-42c9-a342-cd1d1829e7fe,3cad7ddf-2726-42c9-a342-cd1d1829e7fe
1,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,2,4a8e55b0-65a3-4173-af9e-a23365a33ff9,4a8e55b0-65a3-4173-af9e-a23365a33ff9
2,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,3,4e97c067-57ec-4f97-99e5-82202a13858a,4e97c067-57ec-4f97-99e5-82202a13858a
3,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,3,12c6f440-e0ac-47c0-b2f3-503ca86b2d03,12c6f440-e0ac-47c0-b2f3-503ca86b2d03
4,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,1,9f385986-55b5-455a-b6a6-4875fd595880,9f385986-55b5-455a-b6a6-4875fd595880
5,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,3,e8d8b71d-f138-4b2a-a1f0-e1fb50e9f7d9,e8d8b71d-f138-4b2a-a1f0-e1fb50e9f7d9
6,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,2,9f271c16-0e33-4d6e-949e-955a33c1744d,9f271c16-0e33-4d6e-949e-955a33c1744d
7,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,1,caba69da-dd2c-4bd7-9fa4-b1726e097f10,caba69da-dd2c-4bd7-9fa4-b1726e097f10
8,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,1,539d78d9-f850-41c7-8c56-5235ee117806,539d78d9-f850-41c7-8c56-5235ee117806
9,0,b0371a6e-f986-432b-9cd2-1327e8aacaee,L31KFI,yes,no,no,no,primary,no,no,...,---,---,---,---,---,---,---,1,6fcc1cb3-3476-4882-9c43-71a274563ddb,6fcc1cb3-3476-4882-9c43-71a274563ddb


In [38]:
job_firm_js.columns

RangeIndex(start=0, stop=954, step=1)

In [24]:
len(cols)
job_firm_js.columns.difference(merged.columns)
job_firm_js.columns
merged.columns

Index(['JS-age', 'JS-gender', 'JS-highest_edu_level', 'JS-nationality_calc',
       'JS-will_work_night_shift', 'number_x', 'caseid_x', 'JS-name',
       'JS-first_name', 'JS-last_name',
       ...
       'JOB-age_accepted', 'JOB-coworker_gender', 'JOB-coworker_nationality',
       'JOB-manager_gender', 'JOB-nonsmoker_required', 'parent_case_id_y',
       'number__0_y', 'number__1_y', 'JOB-caseid', 'pk'],
      dtype='object', length=518)

In [16]:
cols = (list(job_firm.columns) + list(job_seekers.columns))
job_firm_js.columns = cols
job_firm_js['employed'] = False

In [17]:
#set(job_firm_js[''].values)
#merged['caseid_x']

#merged_x = jobseeker
#merged_y = match
#merged_JS-caseid = jobseeker
#job_firm_jx_x = job-opening
#job_firm_js_y = firm
#job_firm_js_JS-caseid = jobseeker
merged['pk'] = merged['JOB-caseid'] + merged['JS-caseid']
job_firm_js['pk'] = job_firm_js['JOB-caseid'] + job_firm_js['JS-caseid']

In [18]:
job_firm_js['pk'].head(10)

0    b0371a6e-f986-432b-9cd2-1327e8aacaeed5398722-b...
1    b0371a6e-f986-432b-9cd2-1327e8aacaee6d6fd04c-6...
2    b0371a6e-f986-432b-9cd2-1327e8aacaee224ab456-e...
3    b0371a6e-f986-432b-9cd2-1327e8aacaee54dbb7c3-d...
4    b0371a6e-f986-432b-9cd2-1327e8aacaeeab87b426-b...
5    b0371a6e-f986-432b-9cd2-1327e8aacaeeeb9117de-d...
6    b0371a6e-f986-432b-9cd2-1327e8aacaee670c2e42-f...
7    b0371a6e-f986-432b-9cd2-1327e8aacaeea00eabe0-0...
8    b0371a6e-f986-432b-9cd2-1327e8aacaee225b6a21-1...
9    b0371a6e-f986-432b-9cd2-1327e8aacaeea18d0811-f...
Name: pk, dtype: object

In [19]:
on = ['pk']
merged_after = job_firm_js.merge(merged[on], on=on, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', 1)
merged_after = merged_after.append(merged)

ValueError: Plan shapes are not aligned

In [ ]:
formatted = pd.DataFrame()
dvs = ['employed', 'quit', 'fired', 'hired_yes_no']
for col in all_columns:
    if col not in dvs:
        formatted[col] = merged[col]

for col in all_columns:
    if col not in scalar_columns and col not in dvs:
        formatted = one_hot_encode(formatted, col)

            
formatted.to_csv('../../match-data/match-v1/formatted.csv')

In [ ]:
outcomes = pd.DataFrame()
outcomes['retained'] = merged['employed'] & ~(merged['quit'] | merged['fired'])
outcomes['hired'] = merged['employed']
y = pd.DataFrame()
o = []
for index, row in outcomes.iterrows():
    if row['hired'] and row['retained']:
        o.append(2)
    elif not row['hired'] and not row['retained']:
        o.append(0)
    elif row['hired'] and not row['retained']:
        o.append(1)

y['outcomes'] = outcomes['hired']

In [ ]:
to_drop = []
for col in formatted.columns:
    if col in scalar_columns:
        formatted[col] = formatted[col].apply(try_float)
        mean = formatted[col].mean()
        formatted[col] = formatted[col].replace(['---', ''], mean)
        formatted[col] = formatted[col].fillna(mean)
    elif col != 'case_id':
        formatted[col] = formatted[col].fillna(0)
        formatted[col] = formatted[col].astype(int)
        formatted[col] = formatted[col].replace(['---', ''], 0)

    if col.endswith('---') or col.endswith('nan') or col.endswith('-'):
        to_drop.append(col)

formatted = formatted.drop(columns=to_drop)

In [ ]:
y=y.astype('int')
y.to_csv('../../match-data/match-v1/y.csv')

In [ ]:
formatted.to_csv('../../match-data/match-v1/X.csv')
X = formatted

In [ ]:
model = LogisticRegressionCV(max_iter=1000, solver='liblinear', penalty='l1')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
plot_roc(X_test, y_test, model)

In [ ]:
cols = X_test.columns
coef_dict = {}
for coef, feat in zip(model.coef_[0], cols):
    coef_dict[feat] = coef
    
c = {}
for k, v in coef_dict.items():
    c[k] = [v]
    

coef_frame = pd.DataFrame.from_dict(c)
sorted_frame = coef_frame.columns[coef_frame.ix[coef_frame.last_valid_index()].argsort()]
coef_frame.to_csv('../../match-data/match-v1/rfe-coefs.csv')

odds_ratios = np.exp(coef_frame)
scores, pvalues = chi2(X, y)

p_dict = {}
for pvalue, feat in zip(pvalues, cols):
    p_dict[feat] = pvalue
    
c = {}
for k, v in p_dict.items():
    c[k] = [v]
    

pval_frame = pd.DataFrame.from_dict(c)
sorted_frame = pval_frame.columns[pval_frame.ix[pval_frame.last_valid_index()].argsort()]
pval_frame.to_csv('../../match-data/match-v1/rfe-pvalues.csv')

combined = coef_frame.append(pval_frame).append(np.exp(odds_ratios))
combined.to_csv('../../match-data/match-v1/rfe-combined.csv')

In [ ]:
model = LogisticRegressionCV(max_iter=1000, penalty='l1')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
plot_roc(X_test, y_test, model)